<a href="https://colab.research.google.com/github/qinwenshuo/CVAE-linguistic-transfer/blob/main/2_machamp_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# use T4/V100 GPU

# There is always one essay in the ONAC corpus that is causing runtime error. I cann't locate
# that file because it changes each time I parse the corpus. Here is the solution:
# You can start with any big number of concatenation_batch like 500.
# You will encounter error: Invalid input(likely to be caused by GPU out of memory),
# just change the concatenation_batch to a small number like 50 and run again(make
# sure you archieved the parsed results before run again). You will still encounter
# that error, but no worry, you can just repeat the process, change a small number
# and run again until there is only one essay left

concatenation_batch = 500

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
from tqdm import tqdm
import shutil

Mounted at /content/drive


In [ ]:
explanation_dict = {
    'Arabic': 'ar',
    'English': 'en',
    'Spanish': 'es',
    'Croatian': 'hr',
    'Czech': 'cs',
    'German': 'de',
    'Italian': 'it',
    'Norwegian': 'no',
    'Portuguese': 'pt',
    'Latvian': 'lv',
    'Icelandic': 'is',
    'Finnish': 'fi',
    'Chinese': 'zh',
    'Korean': 'ko'
}
print("Please choose one language from the list:", list(explanation_dict.keys()))
language = input("Enter your choice: ")
nativeness = input("Native or Learner?\nEnter your choice: ")
input_directory = f'/content/drive/MyDrive/cvae_project/2_conllu_format/{language}/{nativeness}/'
subdirectories = [name.replace('.zip', '') for name in os.listdir(input_directory) if name.endswith('.zip')]
print("Please choose one corpus from the list:", subdirectories)
corpus = input("Enter your choice: ")
input_directory = f'{input_directory}{corpus}.zip'
emb = 'mbert'
seed = '1'
device = '0'

lg = explanation_dict[language]
lg_dict = {'ar': 'ar_padt', 'en': 'en_ewt', 'es': 'es_ancora', 'hr': 'hr_set', 'cs': 'cs_pdt', 'de': 'de_gsd', 'it':'it_partut', 'no': 'no_bokmaal', 'pt': 'pt_gsd', 'lv': 'lv_lvtb', 'is': 'is_modern', 'fi': 'fi_tdt', 'zh': 'zh_gsdsimp', 'ko': 'ko_kaist'}
lg_code = lg_dict[lg]

archived_output_directory = f'/content/drive/MyDrive/cvae_project/3_pred_format/{language}/{nativeness}/'
output_directory = f'/content/{corpus}/'
logs_dir = f"/content/drive/MyDrive/crosslinguistic_nli/machamp/logs/{lg_code}/{emb}_{seed}/2022.03.09_21.47.34/model.tar.gz"
predict_dir = '/content/drive/MyDrive/crosslinguistic_nli/machamp/predict.py'
os.makedirs(archived_output_directory, exist_ok=True)
os.makedirs(output_directory, exist_ok=True)
archived_output_directory = archived_output_directory + f'{corpus}_machamp_{emb}_{seed}'
print()
print(f'input_directory: {input_directory}')
print(f'output_directory: {output_directory}')
print(f'archived_output_directory: {archived_output_directory}')
print(f'logs_dir: {logs_dir}')
print(f'predict_dir: {predict_dir}')

Please choose one language from the list: ['Arabic', 'English', 'Spanish', 'Croatian', 'Czech', 'German', 'Italian', 'Norwegian', 'Portuguese', 'Latvian', 'Icelandic', 'Finnish', 'Chinese', 'Korean']
Enter your choice: English
Native or Learner?
Enter your choice: Learner
Please choose one corpus from the list: ['WriCLE', 'ArabCC', 'ICNALE', 'ICLE', 'PELIC', 'TOEFL', 'WriCLE_informal', 'BAWE', 'Gachon', 'CLC']
Enter your choice: CLC

input_directory: /content/drive/MyDrive/cvae_project/2_conllu_format/English/Learner/CLC.zip
output_directory: /content/CLC/
archived_output_directory: /content/drive/MyDrive/cvae_project/3_pred_format/English/Learner/CLC_machamp_mbert_1
logs_dir: /content/drive/MyDrive/crosslinguistic_nli/machamp/logs/en_ewt/mbert_1/2022.03.09_21.47.34/model.tar.gz
predict_dir: /content/drive/MyDrive/crosslinguistic_nli/machamp/predict.py


In [ ]:
!pip install overrides

  Preparing metadata (setup.py) ... done
  Created wheel for overrides: filename=overrides-3.1.0-py3-none-any.whl size=10172 sha256=7c0bd95bae7294644af3629404c0669c464d788abacdb3ea4d132547819401fd
  Stored in directory: /root/.cache/pip/wheels/bd/23/63/4d5849844f8f9d32be09e1b9b278e80de2d8314fbf1e28068b
Successfully built overrides


In [ ]:
!pip install allennlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 29.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 34.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8

In [ ]:
# delete unnecessary temporary directory
def delete_directory(directory):
    for root, dirs, files in os.walk(directory, topdown=False):
        for file in files:
            file_path = os.path.join(root, file)
            os.remove(file_path)
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            os.rmdir(dir_path)
    os.rmdir(directory)


In [ ]:
# unzip input files
input_extended_essays_path = f'/content/concatenated_essays/{corpus}'
input_extraction_path = f'/content/original_essays/{corpus}'

shutil.unpack_archive(input_directory, input_extraction_path, 'zip')
os.makedirs(input_extended_essays_path, exist_ok=True)

# unzip parsed files so that you can continue parsing
splitted_output = output_directory + 'splitted/'
os.makedirs(splitted_output, exist_ok=True)

if os.path.exists(archived_output_directory+'.zip'):
    shutil.unpack_archive(archived_output_directory+'.zip', splitted_output, 'zip')


# concatenate short essays into long essays before parsing
count = 0
total_count = 0
all_content = ''
essay_breakers = []
essay_breaker = {}
start_index = 0
out_index = 0
parsed_list = os.listdir(splitted_output)

for file_name in tqdm(os.listdir(input_extraction_path)):
    if file_name.endswith('.conllu') and file_name.replace('.conllu', '.pred') not in parsed_list:
        count += 1
        with open(f'{input_extraction_path}/{file_name}', 'r') as f:
            file_content = f.read()
        line_count = file_content.count("\n")
        essay_breaker[file_name] = (start_index, start_index+line_count+2)
        start_index = start_index + line_count + 2
        all_content = all_content + file_content + '\n' + '\n'

        if count == concatenation_batch:
            total_count += count
            count = 0
            out_index += 1
            essay_breakers.append(essay_breaker)
            essay_breaker = {}
            with open(f'{input_extended_essays_path}/{out_index}.conllu', 'w') as f:
                f.write(all_content)
            all_content = ''
            start_index = 0

if count != 0:
    total_count += count
    out_index += 1
    essay_breakers.append(essay_breaker)
    with open(f'{input_extended_essays_path}/{out_index}.conllu', 'w') as f:
        f.write(all_content)

print(f'\n{total_count} number of essays will be parsed')

100%|██████████| 2481/2481 [00:01<00:00, 1756.71it/s]


2481 number of essays will be parsed


In [ ]:
delete_directory('/content/original_essays/')

In [ ]:
def extract_substring_between_newlines(text, n, m):

    lines = text.split('\n')

    if n < 0 or m < n or m > len(lines):
        raise ValueError(f"Invalid input(likely to be caused by GPU out of memory): n: {n}, m: {m}, length of text: {len(lines)}")

    substring = '\n'.join(lines[n:m])
    return substring

# Example usage
# input_text = "Line 1\nLine 2\nLine 3\nLine 4\nLine 5\nLine 6\nLine 7"
# n_value = 2
# m_value = 5
# result = extract_substring_between_newlines(input_text, n_value, m_value)
# print(result)


In [ ]:
output_path = output_directory + 'concatenated/'
os.makedirs(output_path, exist_ok=True)

In [ ]:
# parsing.... takes about 2 to 40 mins each corpora dependending on the size
# remember to check the error message that may say GPU out of memory

for filename in tqdm(os.listdir(input_extended_essays_path), desc='Processing'):
    out_name = filename.replace('.conllu', '.pred')
    in_dir = os.path.join(input_extended_essays_path, filename)
    out_dir = os.path.join(output_path, out_name)
    !python3 {predict_dir} {logs_dir} {in_dir} {out_dir} --device 0 --batch_size 16 --device {device}
    index = int(filename.replace('.conllu', ''))
    essay_breaker = essay_breakers[index-1]
    with open(output_path+f'{index}.pred', 'r') as f:
        file_content = f.read()
    for key, value in essay_breaker.items():
        essay = extract_substring_between_newlines(file_content, value[0], value[1])
        if essay.startswith('1\t') == False:
            print(essay)
            raise ValueError(f'Incorrect segmentation of essay {key}, {value} in {index}.pred')
        with open(splitted_output+key.replace('.conllu', '.pred'), 'w') as f:
            f.write(essay)


Processing:   0%|          | 0/5 [00:00<?, ?it/s]

2023-08-20 21:18:52.044706: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-20 21:18:55.484649: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-20 21:19:20,817 - INFO - allennlp.models.archival - loading archive file /content/drive/.shortcut-targets-by-id/1juATzWwdx5yPo8nTk3MDAmvLzh78gduZ/crosslinguistic_nli/machamp/logs/en_ewt/mbert_1/2022.03.09_21.47.34 from cache at /content/drive/.shortcut-targets-by-id/1juATzWwdx5yPo8nTk3MDAmvLzh78gduZ/crosslinguistic_nli/machamp/logs/en_ewt/mbert_1/2022.03.09_21.47.34
2023-08-20 21:19:20,864 - INFO - allennlp.common.params - dataset_reader.type = machamp_universal_reader
2023-08-20 21:19:20,864 - INFO - allennlp.common.params - dataset_reader.t

Processing:  20%|██        | 1/5 [02:14<08:59, 134.94s/it]

2023-08-20 21:21:00.627611: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-20 21:21:01.916894: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-20 21:21:06,018 - INFO - allennlp.models.archival - loading archive file /content/drive/.shortcut-targets-by-id/1juATzWwdx5yPo8nTk3MDAmvLzh78gduZ/crosslinguistic_nli/machamp/logs/en_ewt/mbert_1/2022.03.09_21.47.34 from cache at /content/drive/.shortcut-targets-by-id/1juATzWwdx5yPo8nTk3MDAmvLzh78gduZ/crosslinguistic_nli/machamp/logs/en_ewt/mbert_1/2022.03.09_21.47.34
2023-08-20 21:21:06,100 - INFO - allennlp.common.params - dataset_reader.type = machamp_universal_reader
2023-08-20 21:21:06,101 - INFO - allennlp.common.params - dataset_reader.t

Processing:  40%|████      | 2/5 [03:33<05:05, 101.77s/it]

...

Processing:  60%|██████    | 3/5 [04:51<03:01, 90.84s/it] 

2023-08-20 21:23:36.936891: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-20 21:23:38.087074: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-20 21:23:42,694 - INFO - allennlp.models.archival - loading archive file /content/drive/.shortcut-targets-by-id/1juATzWwdx5yPo8nTk3MDAmvLzh78gduZ/crosslinguistic_nli/machamp/logs/en_ewt/mbert_1/2022.03.09_21.47.34 from cache at /content/drive/.shortcut-targets-by-id/1juATzWwdx5yPo8nTk3MDAmvLzh78gduZ/crosslinguistic_nli/machamp/logs/en_ewt/mbert_1/2022.03.09_21.47.34
2023-08-20 21:23:42,776 - INFO - allennlp.common.params - dataset_reader.type = machamp_universal_reader
2023-08-20 21:23:42,777 - INFO - allennlp.common.params - dataset_reader.t

Processing:  80%|████████  | 4/5 [06:06<01:24, 84.80s/it]

2023-08-20 21:24:52.451800: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-20 21:24:53.542903: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-20 21:24:57,022 - INFO - allennlp.models.archival - loading archive file /content/drive/.shortcut-targets-by-id/1juATzWwdx5yPo8nTk3MDAmvLzh78gduZ/crosslinguistic_nli/machamp/logs/en_ewt/mbert_1/2022.03.09_21.47.34 from cache at /content/drive/.shortcut-targets-by-id/1juATzWwdx5yPo8nTk3MDAmvLzh78gduZ/crosslinguistic_nli/machamp/logs/en_ewt/mbert_1/2022.03.09_21.47.34
2023-08-20 21:24:57,096 - INFO - allennlp.common.params - dataset_reader.type = machamp_universal_reader
2023-08-20 21:24:57,096 - INFO - allennlp.common.params - dataset_reader.t

Processing: 100%|██████████| 5/5 [07:24<00:00, 88.95s/it]


In [ ]:
# archive output files to speed up transmission through API
if os.path.exists(archived_output_directory+'.zip'):
    os.remove(archived_output_directory+'.zip')
shutil.make_archive(archived_output_directory, 'zip', splitted_output)

'/content/drive/MyDrive/cvae_project/3_pred_format/English/Learner/CLC_machamp_mbert_1.zip'

In [ ]:
# Call the function to clear the /content/ directory after using the unzipped files
def clear_content_directory():
    content_dir = '/content/'
    exempt_folders = ['drive', 'sample_data']

    for item in os.listdir(content_dir):
        item_path = os.path.join(content_dir, item)
        if os.path.isdir(item_path) and item not in exempt_folders:
            for root, dirs, files in os.walk(item_path, topdown=False):
                for file in files:
                    file_path = os.path.join(root, file)
                    os.remove(file_path)
                for dir in dirs:
                    dir_path = os.path.join(root, dir)
                    os.rmdir(dir_path)
            os.rmdir(item_path)
        elif os.path.isfile(item_path):
            os.remove(item_path)

In [ ]:
# clear temporary /content/ dir
clear_content_directory()